In [1]:
import cv2
import pandas as pd
from moviepy.editor import VideoFileClip
from deepface import DeepFace

def analyze_emotions_and_display(video_path, time_ranges, output_video_path="output_video.mp4"):
    """
    動画全体を再生しながら、指定した時間帯で感情分析を行い、その結果を表示する関数。
    :param video_path: 動画ファイルのパス
    :param time_ranges: 感情分析を行う時間帯のリスト [(start_time1, end_time1), (start_time2, end_time2), ...]
                        時間は秒単位で指定
    :param output_video_path: 出力する動画ファイルのパス
    """
    video = VideoFileClip(video_path)
    width, height = int(video.w), int(video.h)

    # 動画出力の設定（動画ファイル保存用）
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 出力動画フォーマット
    output_video = cv2.VideoWriter(output_video_path, fourcc, video.fps, (width, height))

    # 動画全体の長さを取得
    total_duration = video.duration

    # 時間帯リストを1秒刻みの時間リストに変換（感情分析を行う秒単位のリストを生成）
    analysis_times = []
    for start_time, end_time in time_ranges:
        analysis_times.extend(range(int(start_time), int(end_time) + 1))

    current_time = 0

    while current_time < total_duration:
        # 動画の指定時間にジャンプしてフレームを取得
        video_frame = video.get_frame(current_time)
        # BGR形式に変換（OpenCV用）
        video_frame = cv2.cvtColor(video_frame, cv2.COLOR_RGB2BGR)
        
        # 現在のフレームの時間が感情分析の時間帯に含まれている場合のみ感情分析を実行
        if int(current_time) in analysis_times:
            try:
                # YOLO を使用するように設定
                emotions_list = DeepFace.analyze(video_frame, actions=['emotion'], enforce_detection=False, detector_backend='yolov8')

                # 複数の顔がある場合、結果はリストとして返される
                if not isinstance(emotions_list, list):
                    emotions_list = [emotions_list]

                # 各顔の感情分析結果をもとにフレームに描画
                for emotions in emotions_list:
                    face_region = emotions['region']
                    dominant_emotion = emotions['dominant_emotion']

                    # 顔の領域に矩形を描画
                    x, y, w, h = face_region['x'], face_region['y'], face_region['w'], face_region['h']
                    cv2.rectangle(video_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    
                    # 支配的な感情ラベルを顔の上に表示
                    cv2.putText(video_frame, f"{dominant_emotion}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            except Exception as e:
                print(f"Error analyzing frame at {current_time}s: {e}")

        # フレームを動画に保存
        output_video.write(video_frame)

        # 動画の次のフレームへ（1秒ごとに進める）
        current_time += 1 / video.fps

    # リソースの解放
    output_video.release()
    print(f"Output video saved as {output_video_path}")

# 実行例
video_path = "C:/Notebook/emotionanalyse/sample2021.mp4"
# 分析したい時間帯を指定（例: 5秒から10秒、15秒から20秒の範囲）
time_ranges = [(4, 30), (31, 60)]

# 感情分析結果を動画に表示して出力
analyze_emotions_and_display(video_path, time_ranges, output_video_path="output_with_emotions.mp4")


Output video saved as output_with_emotions.mp4
